## MAIN SOURCE-LEVEL PAC AND STATS

**This script:**
1. Creates theta-gamma PAC comodulograms for condition (all vertices) for each subject and saves PAC data as numpy array
2. Runs cluster-besed permutation test on PAC data

**OUTCOME: PAC estimates for all vertices for each subject and statistical assessment of the results**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
from utils import check_paths
import pandas as pd
from scipy.io import loadmat
import joblib
import matplotlib.gridspec as gridspec

from pactools import Comodulogram, REFERENCES, raw_to_mask

from mne.channels.layout import find_layout
from functools import partial
from mne.defaults import _handle_default

from mne.viz.topo import _erfimage_imshow_unified, _plot_topo

from mne.viz.utils import (
    _setup_vmin_vmax,
    add_background_image
)
from collections import namedtuple

from mne.stats import permutation_cluster_1samp_test

import scipy
from scipy.stats import zscore

%matplotlib qt

1. PAC analysis per condition per subject

In [2]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
group = 'Y'
subs = os.listdir(os.path.join(eeg_data_dir, group))
tasks = ['_MAIN'] # ['_BL', '_MAIN']
task_stages = ['_plan', '_go'] # ['_plan', '_go']
block_names = ['_baseline', '_adaptation'] # ['_baseline', '_adaptation']

theta_range = np.linspace(4, 8, 20)  # Phase: 4-8 Hz
gamma_range = np.linspace(30, 80, 20)  # Amplitude: 30-80 Hz


In [ ]:
for sub_name in subs:
    print(f"Processing subject: {sub_name}")
    analysis_dir = os.path.join(eeg_data_dir, group, sub_name, 'preproc', 'analysis')
    pac_save_path = os.path.join(analysis_dir, 'source', 'PAC')
    check_paths(pac_save_path)

    for task in tasks:
        for task_stage in task_stages: # task_stages #############################
            tmin = 0.0
            tmax = [0.495 if task_stage == '_plan' else 0.695][0] # 0.495 for '_plan', 0.695 for '_go'
            if task == '_MAIN':
                for block_name in block_names: # block_names #########################
                        stcs_path = os.path.join(analysis_dir, 'source', 'morphed_stcs', task, task_stage, block_name) 

                        stcs = []
                        stcs_data = []

                        for stc_file in os.listdir(stcs_path):
                            if stc_file.endswith('-rh.stc'): # MNE will load both hemispheres anyway
                                stc_path = os.path.join(stcs_path, stc_file)
                                stc = mne.read_source_estimate(stc_path, subject=sub_name)
                                stc.crop(tmin=tmin, tmax=tmax)
                                stcs.append(stc)
                                stcs_data.append(stc.data)

                        source_array = np.stack(stcs_data, axis=0)
                        print(source_array.shape)  # (epochs x vertices x time)

                        times = stcs[0].times

                        #Estimate PAC
                        estimator = Comodulogram(
                            fs=stcs[0].sfreq,
                            low_fq_range=theta_range,  # Phase frequencies (theta)
                            high_fq_range=gamma_range, # Amplitude frequencies (gamma)
                            method='tort',
                            progress_bar=True
                            )

                        pac_results = np.empty(
                            (len(theta_range), source_array.shape[1], len(gamma_range))
                        )

                        for i in range(source_array.shape[1]):
                            print(f"Processing source {i+1}/{source_array.shape[1]}")

                            data_flat = np.reshape(source_array[:, i], -1)[None, :]
                            pac = estimator.fit(
                                    data_flat,
                                    data_flat,
                                )
                            pac_results[:, i] = pac.comod_

                            # if i in range(10):
                            #     # Convert the plot to a Plotly figure (if supported)
                            #     fig = pac.plot(tight_layout=False, cmap='magma')
                            #     # Add a title
                            #     plt.title(f"PAC MI {sub_name[-5:]} - source={i}: {task}{task_stage}{block_name}")

                            #     # Save the plot
                            #     plt.show()

                        np.save(os.path.join(pac_save_path, f"PAC_MI_SOURCE_{sub_name[-5:]}{task}{task_stage}{block_name}.npy"), pac_results)
            else:
                continue

_________________

In [ ]:
# Plotting the first 10 vertices of the first epoch
plt.figure(figsize=(10, 6))

for i in range(10):
    plt.plot(times, source_array[0, i], label=f'Vertex {i}')


____________

# STATISTICS
**Cluster-based permutation test**

In [3]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'

groups = ['Y']
task = '_MAIN' # ['_BL', '_MAIN']
task_stages = ['_plan', '_go']
block_names = ['_baseline', '_adaptation']

**STATS FOR DATA AVERAGED ACROSS PHASE AND AMPLITUDE FREQUENCIES**

**PER CONDITION**

*Looking into imputation of NaN values (optional)*

In [ ]:
# Visualizing the sparsity pattern of the adjacency matrix
source_adjacency = mne.spatial_src_adjacency(src)
adj = source_adjacency.tocsr()
plt.figure(figsize=(6, 6))
plt.spy(adj, markersize=0.5)
plt.title("Sparsity pattern of CSR matrix")
plt.xlabel("Columns")
plt.ylabel("Rows")
plt.show()

plt.figure(figsize=(6, 6))
plt.spy(source_adjacency, markersize=0.5)
plt.title("Sparsity pattern of COO matrix")
plt.xlabel("Columns")
plt.ylabel("Rows")
plt.show()

In [ ]:
### NAN Imputation for PAC Matrices ###
# Step 0: Ensure adjacency is CSR format for fast indexing
adj = source_adjacency.tocsr()  # your source adjacency matrix
pac_imputed = pac_t.copy()  # to preserve original

# Step 1: Detect vertices with any NaN in their 20×20 PAC
nan_mask = np.isnan(pac_t).any(axis=(1, 2))  # shape (5124,)

print(f"Found {nan_mask.sum()} vertices with NaN values.")

# Step 2: Impute NaNs from neighbors
for vtx in np.where(nan_mask)[0]:
    neighbors = adj[[vtx]].indices
    valid_neighbors = [n for n in neighbors if not nan_mask[n]]

    if valid_neighbors:
        # Average PAC matrices from neighbors
        pac_imputed[vtx] = np.nanmean(pac_t[valid_neighbors], axis=0)
    else:
        # Fallback: no valid neighbors, fill with global PAC mean
        pac_imputed[vtx] = np.nanmean(pac_t, axis=0)

print("NaN imputation complete.")


Found 9 vertices with NaN values.
NaN imputation complete.


In [ ]:
# Find the indices of vertices with all-NaN PAC matrices
for sub_name in subs[:9]:

    print(sub_name)
    sub_dir = os.path.join(eeg_data_dir, group, sub_name)
    pac_dir = os.path.join(sub_dir, 'preproc', 'analysis', 'source', 'PAC')

    # Load PAC data
    pac = np.load(os.path.join(pac_dir, f"PAC_MI_SOURCE_{sub_name[-5:]}{task}{task_stage}{block_name}.npy"))
    pac_t = np.transpose(pac, (1, 0, 2))
    nan_count = np.isnan(pac_t).sum()
    print(f"Total NaN values in pac_t: {nan_count}")

    # Create a boolean mask for vertices with all-NaN PAC matrices
    all_nan_vertices = np.isnan(pac_t).all(axis=(1, 2))  # shape: (5124,)

    # Get indices of those vertices
    nan_vertex_indices = np.where(all_nan_vertices)[0]

    # Count them
    print(f"Number of vertices with all-NaN PAC: {len(nan_vertex_indices)}")
    print(f"Indices: {nan_vertex_indices}")

s1_pac_sub01
Total NaN values in pac_t: 2000
Number of vertices with all-NaN PAC: 5
Indices: [2431 3457 4029 4454 4562]
s1_pac_sub07
Total NaN values in pac_t: 1200
Number of vertices with all-NaN PAC: 3
Indices: [1648 4345 4968]
s1_pac_sub10
Total NaN values in pac_t: 1200
Number of vertices with all-NaN PAC: 3
Indices: [ 161  350 2431]
s1_pac_sub11
Total NaN values in pac_t: 5200
Number of vertices with all-NaN PAC: 13
Indices: [ 332 1360 1433 2359 2602 2956 3183 3340 3364 3384 3394 3570 5069]
s1_pac_sub22
Total NaN values in pac_t: 3600
Number of vertices with all-NaN PAC: 9
Indices: [  70 1457 1696 1993 2197 3327 3391 4562 4863]
s1_pac_sub24
Total NaN values in pac_t: 6800
Number of vertices with all-NaN PAC: 17
Indices: [ 341  956 1330 1331 1425 1738 1878 1990 2259 2431 2859 2922 3114 4035
 4233 4699 4865]
s1_pac_sub26
Total NaN values in pac_t: 6000
Number of vertices with all-NaN PAC: 15
Indices: [ 678 1736 2361 2622 2835 3703 3742 3777 3783 3787 4205 4437 4762 4780
 5086]
s1_pa

In [4]:
# Main script to process PAC data and run cluster-based permutation tests

############### CREATE ADJACENCY MATRIX FOR STATISTICAL TEST #############
# find_ch_adjacency first attempts to find an existing "neighbor"
# (adjacency) file for given sensor layout.
# If such a file doesn't exist, an adjacency matrix is computed on the fly,
# using Delaunay triangulations.
src_fname = 'D:\\BonoKat\\research project\\# study 1\\mri_data\\fs_output\\freesurfer\\sub_dir\\Y\\fsaverage_bem\\bem\\fsaverage-ico4-src.fif'
src = mne.read_source_spaces(src_fname)
# src.plot(subjects_dir='D:\\BonoKat\\research project\\# study 1\\mri_data\\fs_output\\freesurfer\\sub_dir\\Y')
source_adjacency = mne.spatial_src_adjacency(src)
adj = source_adjacency.tocsr()  # Ensure adjacency is CSR format for fast indexing
print('adjacency shape:', source_adjacency.shape)


############## PROCESS PAC DATA FOR EACH GROUP AND TASK STAGE #############
for group in groups:
    group_save_path = os.path.join(eeg_data_dir, f'{group} group')
    pac_stats_save_path = os.path.join(group_save_path, 'source_pac_stats')
    check_paths(pac_stats_save_path)
    
    # # Create directories for saving figures
    # fig_group_path = os.path.join(pac_stats_save_path, 'figs')
    # fig_group_save_path = os.path.join(fig_group_path, group)
    # fig_task_save_path = os.path.join(fig_group_path, group, task)
    # check_paths(fig_group_path, fig_group_save_path, fig_task_save_path)

    subs = os.listdir(os.path.join(eeg_data_dir, group))

    for task_stage in task_stages: # [task_stages[0]]
        for block_name in block_names: # [block_names[0]]

            print(f'Processing {group} group, {task} task, {task_stage} stage, {block_name} block...')

            ############# STACK PAC DATA OF INDIVIDUAL PARTICIPANTS #############

            # Create a list to store the PAC data for each subject
            pac_list = []
            pac_zscore_list = []

            for sub_name in subs:

                sub_dir = os.path.join(eeg_data_dir, group, sub_name)
                pac_dir = os.path.join(sub_dir, 'preproc', 'analysis', 'source', 'PAC')

                # Load PAC data
                pac = np.load(os.path.join(pac_dir, f"PAC_MI_SOURCE_{sub_name[-5:]}{task}{task_stage}{block_name}.npy"))
                pac_t = np.transpose(pac, (1, 0, 2))

                ### NAN Imputation for PAC Matrices ###
                pac_imputed = pac_t.copy()

                # Step 1: Detect vertices with any NaN in their 20×20 PAC
                nan_mask = np.isnan(pac_t).any(axis=(1, 2))  # shape (5124,)

                if nan_mask.any() == True:
                    print(f"Found {nan_mask.sum()} vertices with NaN values.")

                    # Step 2: Impute NaNs from neighbors
                    for vtx in np.where(nan_mask)[0]:
                        neighbors = adj[[vtx]].indices
                        valid_neighbors = [n for n in neighbors if not nan_mask[n]]
                        # Average PAC matrices from neighbors
                        pac_imputed[vtx] = np.nanmean(pac_t[valid_neighbors], axis=0)

                    print(f"NaN imputation for {sub_name} complete.")

                pac_list.append(pac_imputed)
                pac_zscore_list.append(zscore(pac_imputed, axis=0, nan_policy='omit')) # 'omit' ignores NaN values in the z-score calculation

            # Stack them along a new first axis (subject axis)
            pac_all = np.stack(pac_list, axis=0)

            # Z-score the PAC data across subjects
            pac_zscore_all = np.stack(pac_zscore_list, axis=0)
            # pac_zscore_all = zscore(pac_all, axis=1, nan_policy='omit') # another way to zscore the data
            print('PAC array shape:', pac_all.shape) # subs x electrodes x ph_freqs x amp_freqs
            print('z-scored PAC array shape:', pac_zscore_all.shape)

            # Averafe z-scored PAC over phase and amplitude frequencies
            # pac_zscore_all_ave = np.mean(pac_zscore_all, axis=(2, 3)) # (23, 5124) subs x electrodes
            # pac_zscore_all_med = np.median(pac_zscore_all, axis=(2, 3)) # produces more significant clusters

            ### Global normalization
            pac_all_ave = np.mean(pac_all, axis=(2, 3))
            pac_zscore_all_ave = (pac_all_ave - np.nanmean(pac_all_ave)) / np.nanstd(pac_all_ave)
            ###

            # # Save the PAC data
            np.save(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{group}{task}{task_stage}{block_name}_ZSCORE_freqs_ave.npy"), pac_zscore_all_ave)

            # # ############# PLOT AND SAVE Z-SCORED PAC AVERAGED ACROSS PARTICIPANTS #############
            # pac_plot, ax1 = plot_rect_topo_from_epochs(np.mean(pac_zscore_all_ave, axis=(0)), epochs.info,
            #                                         title=f'{group}{task}{task_stage}{block_name}: Averaged z-scored PAC MI',
            #                                         cmap='PiYG', vmin=-0.5, vmax=0.5)
            # plt.savefig(os.path.join(fig_task_save_path, f"pac_mi_{group}{task}{task_stage}{block_name}_PAC_MI_AVE_TOPO.png"), dpi=300)

            # pac_ave_plot, ax2 = plot_matrix_topo_from_epochs(np.mean(pac_zscore_all, axis=(0)), epochs.info,
            #                                                 title=f'{group}{task}{task_stage}{block_name}: z-scored PAC MI',
            #                                                 cmap='PiYG', vmin=-0.5, vmax=0.5)
            # plt.savefig(os.path.join(fig_task_save_path, f"pac_mi_{group}{task}{task_stage}{block_name}_PAC_MI_TOPO.png"), dpi=300)


            ############# RUN CLUSTER-BASED PERMUTATION TEST #############

            tail = 0 # two-tailed test

            # Set the threshold for including data bins in clusters with t-value corresponding to p=0.01
            # Because we conduct a two-tailed test, we divide the p-value by 2 (which means we're making use of both tails of the distribution).
            # As the degrees of freedom, we specify the number of observations (here: subjects) minus 1.
            # Finally, we subtract 0.01 / 2 from 1, to get the critical t-value on the right tail
            p_threshold = 0.01
            degrees_of_freedom = pac_all.shape[0] - 1
            t_thresh = scipy.stats.t.ppf(1 - p_threshold / 2, df=degrees_of_freedom)

            #!
            # threshold_tfce = dict(start=0, step=0.2) # Threshold-free cluster enhancement (TFCE) - more conservative, similar results

            # Set the number of permutations
            n_permutations = 10000

            # Run the analysis
            T_obs, clusters, cluster_p_values, H0 = permutation_cluster_1samp_test(
                pac_zscore_all_ave,
                n_permutations=n_permutations,
                threshold=t_thresh, # None - default threshold based on t-distribution
                tail=tail,
                adjacency=adj,
                step_down_p=0.05,  # step-down p-value correction instead of max stats
                out_type="mask",
                max_step=1,
                n_jobs=-1 # to use all available CPU cores
            )

            # # Save the results
            np.save(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{group}{task}{task_stage}{block_name}_freqs_ave_T_obs.npy"), T_obs)
            np.save(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{group}{task}{task_stage}{block_name}_freqs_ave_clusters.npy"), np.array(clusters, dtype=object))
            np.save(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{group}{task}{task_stage}{block_name}_freqs_ave_cluster_p_values.npy"), cluster_p_values)
            np.save(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{group}{task}{task_stage}{block_name}_freqs_ave_H0.npy"), H0)

            # SANITY CHECKS
            print(f't_thresh = {t_thresh}')
            print(f'T_obs_mean = {T_obs.mean()}')
            print(f'cluster_p_values = {cluster_p_values}')

            alpha = 0.05  # significance threshold
            significant_clusters = [i for i, p in enumerate(cluster_p_values) if p < alpha]
            print(f"Condition {task_stage}{block_name}: Found {len(significant_clusters)} significant clusters out of {len(cluster_p_values)} total clusters.")


            # ####### PLOT THE RESULTS #######
            # plot_significant_topomap(T_obs, clusters, cluster_p_values, epochs.info, group=group, task=task, task_stage=task_stage, block_name=block_name)
            # plt.savefig(os.path.join(fig_task_save_path, f"pac_cluster_stats_{group}{task}{task_stage}{block_name}_freq_ave_TOPO.png"), dpi=300)

    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    2 source spaces read
-- number of adjacent vertices : 5124
adjacency shape: (5124, 5124)
Processing Y group, _MAIN task, _plan stage, _baseline block...
Found 5 vertices with NaN values.
NaN imputation for s1_pac_sub01 complete.
Found 3 vertices with NaN values.
NaN imputation for s1_pac_sub07 complete.
Found 3 vertices with NaN values.
NaN imputation for s1_pac_sub10 complete.
Found 13 vertices with NaN values.
NaN imputation for s1_pac_sub11 complete.
Found 9 vertices with NaN values.
NaN imputation for s1_pac_sub22 complete.
Found 17 vertices with NaN values.
NaN imputation for s1_pac_sub24 complete.
Found 15 vertices with NaN values.
NaN imputation for s1_pac_sub26 complete.
Found 17 vertices with NaN values.
NaN imputation for s1_pac_sub29 complete.
Found 9 vertices wi

  0%|          | Permuting : 0/9999 [00:00<?,       ?it/s]

Step-down-in-jumps iteration #1 found 1 cluster to exclude from subsequent iterations


  0%|          | Permuting : 0/9999 [00:00<?,       ?it/s]

Step-down-in-jumps iteration #2 found 0 additional clusters to exclude from subsequent iterations
t_thresh = 2.818756060596369
T_obs_mean = -0.10162464930701287
cluster_p_values = [1.     0.9999 0.9614 1.     1.     0.8425 1.     0.9999 0.9585 0.6849
 0.0248 0.8041 1.     0.6853 0.9148 0.945  1.     0.8051 1.     0.6949
 0.9636 1.     1.     0.8644 0.9998 0.6903 1.     1.     1.     1.
 1.     0.8573 0.9807 1.     0.4921 1.     1.     0.8854 0.9833 1.
 1.     0.9307 1.     0.2    0.3442 1.     0.9978 1.     1.     0.9983
 1.     0.8935 1.     0.8322 1.     0.9825 1.     0.9741 0.8386 0.9081
 0.9974 1.     1.     0.9857 1.     1.    ]
Condition _plan_baseline: Found 1 significant clusters out of 66 total clusters.
Processing Y group, _MAIN task, _plan stage, _adaptation block...
Found 5 vertices with NaN values.
NaN imputation for s1_pac_sub01 complete.
Found 3 vertices with NaN values.
NaN imputation for s1_pac_sub07 complete.
Found 3 vertices with NaN values.
NaN imputation for s1_pac

...

Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
t_thresh = 2.818756060596369
T_obs_mean = -0.2586599887322655
cluster_p_values = [0.8063 0.7024 0.9699 0.8826 0.7997 0.9966 0.8775 0.3162 0.2186 0.6029
 0.7018 0.9957 0.78   0.8888 0.6846 0.4211 0.8821 0.9328 0.9715 0.6804
 0.9094 0.654  0.6659 0.6713 0.7502 0.6147 0.9741 0.6517 0.9832 0.8492
 0.8596 0.969  0.9477 0.7944 0.4129 0.9076 0.7979 0.7167 0.868  0.8948
 0.5262 0.4529 0.1771 0.2947 0.6426 0.9699 0.0782 0.9896 0.9624 0.9142
 0.3017 0.9984 0.7712 0.9098 0.8594 0.9328 0.4873 0.2923 0.2746 0.8566
 0.6092 0.3207 0.5196 0.8169 0.8645 0.9604 0.9571 0.657  0.9893 0.9969
 0.7926 0.4289 0.4508 0.1916 0.9283 0.3159 0.7159 0.6031 0.9952 0.9832
 0.7865 0.8263 0.984  0.7753 0.7383]
Condition _plan_adaptation: Found 0 significant clusters out of 85 total clusters.
Processing Y group, _MAIN task, _go stage, _baseline block...
Found 5 vertices with NaN values.
NaN imputation for s1_pac_sub01 complete.
Found 

  0%|          | Permuting : 0/9999 [00:00<?,       ?it/s]

Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
t_thresh = 2.818756060596369
T_obs_mean = -0.15067282161849233
cluster_p_values = [1.     1.     0.855  0.3057 0.4529 0.9997 0.094  0.9863 1.     0.3034
 1.     0.9957 1.     1.     1.     0.9166 0.626  1.     0.9427 0.829
 0.999  0.8576 0.8806 0.9515 1.     1.     1.     0.9252 0.9999 0.9999
 0.9998 0.9978 0.905  0.9927 0.9954 0.9999 0.6125 0.7908 1.     0.8796
 1.     0.9999 0.634  0.9989 1.     1.     1.     0.9999 0.9337 0.9978
 1.     0.7754 0.9998 1.     1.     0.997  0.9959 0.9079 1.     0.9964]
Condition _go_baseline: Found 0 significant clusters out of 60 total clusters.
Processing Y group, _MAIN task, _go stage, _adaptation block...
Found 5 vertices with NaN values.
NaN imputation for s1_pac_sub01 complete.
Found 3 vertices with NaN values.
NaN imputation for s1_pac_sub07 complete.
Found 3 vertices with NaN values.
NaN imputation for s1_pac_sub10 complete.
Found 13 vertices with NaN values.

  0%|          | Permuting : 0/9999 [00:00<?,       ?it/s]

Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
t_thresh = 2.818756060596369
T_obs_mean = -0.22435349356053857
cluster_p_values = [0.9999 0.8681 0.0722 0.4712 0.5572 0.8751 0.5994 0.4936 0.9977 0.9999
 0.477  0.9665 0.9945 0.8064 0.9933 0.9988 0.9987 0.6347 1.     1.
 1.     0.9949 0.4341 0.9984 0.9129 0.9205 0.9032 0.8256 0.9785 0.9967
 0.9973 0.9358 0.9989 0.9546 0.9999 0.9976 0.7807 0.9052 0.7734 0.9994
 0.8735 0.9988 0.9987 0.9299 0.8621 0.9999 0.9996 0.8493 0.9999 0.7787
 0.9869 0.5871 0.2554 0.0946 0.9998 0.9823 0.0548 0.9999 0.994  0.9881
 0.4545 0.7787 0.9966 1.     0.9675 0.9804 0.8832 0.5279 0.9412 0.945
 0.987  0.2929 0.4591 0.9823 0.9179 0.9999 0.8323 0.4265 0.7853 0.9996
 0.994  0.6222 0.7794 0.9984 0.9997 0.9707 0.9716 0.7758 0.9356 0.749
 0.9862 1.     0.8202 0.5237 0.9746 0.9989 0.9989 0.9249 0.9952]
Condition _go_adaptation: Found 0 significant clusters out of 99 total clusters.


__________________

Dirty field

PLOTTING PAC DISTRIBUTIONS

In [ ]:
# Global distribution of all PAC values across all subjects and vertices
# Flatten all PAC values into a 1D array
all_pac_values = pac_all.flatten()

# Remove NaNs if present
all_pac_values = all_pac_values[~np.isnan(all_pac_values)]

# Plot histogram
plt.figure(figsize=(8, 5))
plt.hist(all_pac_values, bins=100, color='skyblue', edgecolor='k')
plt.title("Distribution of all PAC z-scores")
plt.xlabel("PAC z-score")
plt.ylabel("Count")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Distribution per subject (e.g., mean PAC per subject)
# Mean PAC per subject, averaged over all space and frequencies
subject_means = np.nanmean(pac_all, axis=(1, 2, 3))  # shape (23,)

plt.figure(figsize=(8, 5))
plt.hist(subject_means, bins=15, color='coral', edgecolor='k')
plt.title("Mean PAC z-scores per subject")
plt.xlabel("Mean PAC z-score")
plt.ylabel("Count")
plt.grid(True)
plt.tight_layout()
plt.show()


In [21]:
# Violin plot: per-subject PAC distributions
import seaborn as sns

# Sample 5000 random PAC values per subject (if needed for speed)
np.random.seed(42)
subset = [pac_all[i].flatten() for i in range(23)]
subset = [x[~np.isnan(x)] for x in subset]
subset = [np.random.choice(x, 5000, replace=False) if len(x) > 5000 else x for x in subset]

# Create DataFrame for seaborn
import pandas as pd
df = pd.DataFrame({
    'PAC': np.concatenate(subset),
    'Subject': np.concatenate([[f'Subj {i+1}'] * len(x) for i, x in enumerate(subset)])
})

plt.figure(figsize=(12, 6))
sns.violinplot(data=df, x='Subject', y='PAC', inner='box')
plt.title("PAC value distribution per subject")
plt.ylabel("PAC z-score")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


In [18]:
np.min(pac_all)

np.float64(5.687941503192036e-07)